# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

# import statements
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# import ML modules
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
engine = create_engine('sqlite:///DisasterResponseYT.db')
df = pd.read_sql_table('DisasterResponseMaster', engine)
x = df.message
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
words = stopwords.words("english")

def tokenize(text):
    # get list of all urls using regex
    de_urls = re.findall(regex, text)
    
    # replace each url in text string with placeholder
    for i in de_urls:
        text = text.replace(i,"urlplaceholder")
    
    # remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    token = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean = []
    for j in token:
        if j not in words:
            # lemmatize, normalize case, and remove leading/trailing white space
            clean_j = lemmatizer.lemmatize(j).lower().strip()
            clean.append(clean_j)

    return clean

for message in x[:5]:
    token = tokenize(message)
    print(message)
    print(token, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=5)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 12)

# train classifier
pipeline.fit(x_train, y_train)

# predict on test data
y_pred = pipeline.predict(x_test)

In [6]:
type(y_test)

pandas.core.frame.DataFrame

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def get_scores(y_test, y_pred):
    scores = pd.DataFrame(columns=['category', 'f1_score', 'precision', 'recall'])
    for i, cat in enumerate(y_test.columns):
        precision, recall, f1_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,i], average='weighted')
        scores.set_value(i + 1, 'category', cat)
        scores.set_value(i + 1, 'f1_score', f1_score)
        scores.set_value(i + 1, 'precision', precision)
        scores.set_value(i + 1, 'recall', recall)
    print('Average precision:', scores['precision'].mean())
    print('Average recall:', scores['recall'].mean())
    return scores

In [8]:
accuracy = (y_pred == y_test).values.mean()
accuracy

0.94197351914013494

In [9]:
get_scores(y_test, y_pred)

Average precision: 0.929788840759
Average recall: 0.94197351914


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,category,f1_score,precision,recall
1,related,0.780364,0.777858,0.788526
2,request,0.862229,0.86219,0.873512
3,offer,0.99451,0.992992,0.996033
4,aid_related,0.737032,0.73769,0.739701
5,medical_help,0.901613,0.896992,0.920201
6,medical_products,0.937994,0.935569,0.949649
7,search_and_rescue,0.960337,0.958596,0.969637
8,security,0.972308,0.964872,0.97986
9,military,0.956475,0.954,0.966585
10,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa7a73f1048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
               oob_score=False, random_state=None, 

In [11]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 3],
}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = 4, verbose = 2)

In [12]:
cv.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=11.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=11.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=11.7min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, total=10.8min
[CV] clf__estimator__min_samples_split=3, clf__estim

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 54.8min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 54.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [25]:
# predict on test data using the best model
y_cv_pred = cv.predict(x_test) 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
accuracy = (y_cv_pred == y_test).values.mean()
print (accuracy)


0.948530159699


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [28]:
pipeline2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [29]:
# predict on test data
y_pip2_pred = pipeline2.predict(x_test)


accuracy = (y_pip2_pred == y_test).values.mean()
accuracy

0.94736039060115962

### 9. Export your model as a pickle file

In [30]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.